# Описание проекта

Интернет-магазин «Викишоп» запускает новый сервис.
Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. 
То есть клиенты предлагают свои правки и комментируют изменения других. 
Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные.
В вашем распоряжении набор данных с разметкой о токсичности правок.
Постройте модель со значением метрики качества F1 не меньше 0.75.

# Загрузка библиотек

In [2]:
import pandas as pd
import numpy as np
import torch
import transformers
from tqdm import notebook
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

# Загрузка данных

In [3]:
df = pd.read_csv('datasets/toxic_comments.csv')

# Предобработка данных

In [4]:
df.isna().sum()

text     0
toxic    0
dtype: int64

Пропусков в данных нет

In [5]:
df.duplicated().sum()

0

Дубликатов в данных нет

In [12]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

## Вывод

* В текущих данных имеем большой дисбаланс классов
* Дубликатов и пропусков данные не имеют

# Создание модели

Токенизируем наши предложения

In [5]:
tokenizer = transformers.BertTokenizer(vocab_file='model/vocab.txt')

config = transformers.BertConfig.from_json_file(
    'model/bert_config.json')
model = transformers.BertModel.from_pretrained(
    'model/pytorch_model.bin', config=config)

In [6]:
tokenized = df['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True,truncation=True, max_length=512))
tokenized = np.array(tokenized)

Ограничим максимальную длинну токенов

In [7]:
max_len = 512
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized])

Создадим маску,чтобы обозначить модели какие значения стоит считать важными

In [8]:
attention_mask = np.where(padded != 0, 1, 0)

Обозначим, что модель должна обучаться на GPU

In [9]:
device = torch.device("cuda:0")
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

Создадим эмбеддинги

In [11]:
%%time
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to(device)
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).to(device)
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/1595 [00:00<?, ?it/s]

Wall time: 36min 46s


Объединим эмбеддинги в массив признаков

In [12]:
features = np.concatenate(embeddings)

In [21]:
features.shape

(159500, 768)

In [17]:
y = df['toxic']
y =y[:159500]

In [25]:
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2,random_state =42)

Создадим и обучим модель на готовых эмбеддингах полученных из модели bert

In [26]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

C:\Users\A\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [27]:
f1_score(logreg.predict(X_test), y_test)

0.6100500648989431

# Вывод

* На текущих данных применение bert не дает высоких результатов
* Возможно стоит попробовать другие модели классификации на эмбеддингах, скорее всего это улучшит результат